<a href="https://colab.research.google.com/github/Sebas20050700/PIPELINE_AERO-METEREOLOGICO/blob/Visual-Crossing/API_VISUAL_CROSSING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **API VISUAL CROSSING**

In [ ]:
import requests
import pandas as pd

def consultar_clima_aeropuertos(lista_aeropuertos):
    """
    ETAPA 1: Ingesta Primaria.
    Recibe códigos ICAO (ej: SPJC, SPZO) y genera las coordenadas
    maestras para OpenSky y SENAMHI.
    """
    url = "https://visual-crossing-weather.p.rapidapi.com/forecast"
    api_key = "a7117854a0msh861c287680abd70p1dfc59jsnfffd314081ec"

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "visual-crossing-weather.p.rapidapi.com"
    }

    resultados = []

    for icao in lista_aeropuertos:
        # Agregamos ", PE" automáticamente para asegurar precisión y evitar Error 400
        location_query = f"{icao}, PE"

        querystring = {
            "aggregateHours": "1",
            "location": location_query,
            "contentType": "json",
            "unitGroup": "metric",
            "shortColumnNames": "false"
        }

        print(f"🛰️ ETAPA 1: Geolocalizando y consultando clima global para: {location_query}...")

        try:
            response = requests.get(url, headers=headers, params=querystring)
            response.raise_for_status()
            data = response.json()

            # Ubicación detectada por la API
            loc_id = list(data['locations'].keys())[0]
            current = data['locations'][loc_id]['currentConditions']

            # Diccionario de salida: Este es el "contrato" con tus compañeros
            resultados.append({
                "aeropuerto_id": icao,
                "lat": data['locations'][loc_id].get('latitude'), # <--- PARA OPENSKY/SENAMHI
                "lon": data['locations'][loc_id].get('longitude'),# <--- PARA OPENSKY/SENAMHI
                "temp_celsius": current.get('temp'),
                "viento_kmh": current.get('wspd'),
                "rafagas_kmh": current.get('wgust'),
                "visibilidad_km": current.get('visibility'),
                "condicion_global": current.get('conditions'),
                "timestamp_utc": current.get('datetime')
            })

        except Exception as e:
            print(f"❌ Error procesando {icao}: {e}")

    return pd.DataFrame(resultados)


# Ejemplo de uso

In [ ]:
# --- EJECUCIÓN DEL PIPELINE ---

# Definimos la ruta de interés
ruta_aerea = ["SPJC", "SPZO"] # Lima a Cusco

# Disparamos la Etapa 1
df_maestro = consultar_clima_aeropuertos(ruta_aerea)

# Mostrar el "Contrato de Datos" que le pasas a los demás
print("\n--- DATA MAESTRA PARA EL EQUIPO ---")
print(df_maestro[['aeropuerto_id', 'lat', 'lon', 'temp_celsius', 'condicion_global']])

In [ ]:
df_maestro.to_csv("data_maestra_clima.csv", index=False)